<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/Copia_de_1_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1AQr9H9bXDeNPchTRufU78g8z0yxHvrmC" width="100%">

# Expresiones Regulares
---

En este taller guiado presentaremos una introducción práctica a las expresiones regulares y sus aplicaciones, en especial, veremos cómo utilizarlas desde el lenguaje de programación _Python_ con la librería `re`:

In [ ]:
import re

## **1. Conceptos de Expresiones Regulares**
---

Una expresión regular (*regex*) es una secuencia de caracteres que describen un patrón de búsqueda. Se utilizan para búsqueda, sustitución, validación y recuperación de información de manera eficiente a partir de información textual. Generalmente, existen distintos motores de expresiones regulares que están implementados en distintos lenguajes de programación (e.g., Python, JavaScript, Perl, Go, entre otros), bases de datos (e.g., PostgreSQL, MongoDB, BigQuery, entre otros), navegadores web, editores de texto y utilidades base en distintos sistemas operativos basados en Linux en funciones como [`grep`](https://www.gnu.org/software/grep/manual/grep.html) o [`sed`](https://www.gnu.org/software/sed/manual/sed.html).

Las expresiones regulares se pueden entender como una forma compacta de definir conjuntos sobre cadenas de caracteres. Por ejemplo, suponga que deseamos detectar los caracteres que hay en la placa (número de matrícula) de los automóviles en Colombia, las cuales están dadas por tres letras entre la A y la Z, seguidas de tres dígitos (e.g., ABC123).

> ¿De qué forma podríamos definir el conjunto de todas las placas y qué cardinalidad tiene este conjunto?

<img src="https://drive.google.com/uc?export=view&id=1hSSgi9Vlv_5tU1cshKIkkght47g9sAyT" width="60%">

Antes de trabajar con expresiones regulares, pensemos en un enfoque crudo para detectar si una cadena de caracteres pertenece al conjunto de placas colombianas, es decir, vamos a validar si los primeros tres caracteres se encuentran entre las letras A-Z y que los últimos tres caracteres se encuentran entre los dígitos 0-9. Comenzamos importando la variable `ascii_uppercase` que nos permite sacar la lista de letras A-Z y la variable `digits` que nos permite obtener la lista de dígitos 0-9.


In [ ]:
from string import ascii_uppercase, digits
print(ascii_uppercase)
print(digits)

También importamos la función `product` para obtener todas las posibles combinaciones de placas. Note que estamos especificando la forma de las combinaciones como 3 letras en `ascii_uppercase` y 3 dígitos en `digits`:

In [ ]:
from itertools import product
combinations = product(
        ascii_uppercase,
        ascii_uppercase,
        ascii_uppercase,
        digits,
        digits,
        digits
        )

Ahora, mapeamos todas las combinaciones como cadenas de caracteres:

In [ ]:
combinations = ["".join(comb) for comb in combinations]

Veamos las primeras 10 combinaciones de este conjunto de placas:

In [ ]:
print(combinations[:10])

También podemos ver la cardinalidad de este conjunto:

In [ ]:
print(len(combinations))

¡Tenemos más de 17 millones de combinaciones de placas!

Ahora veamos un ejemplo de cómo podríamos validar si una cadena de caracteres pertenece a este conjunto:

In [ ]:
q = "ABC123"
print(q in combinations)

Como podemos ver, la secuencia `ABC123` es una placa válida de automóvil, veamos otro ejemplo con la secuencia `1A2B3C`:

In [ ]:
q = "1A2B3B"
print(q in combinations)

Esta forma de validar placas presenta varios problemas:

* Tiene una alta complejidad en memoria. Debemos almacenar muchas combinaciones de cadenas de caracteres para únicamente evaluar una secuencia de tamaño 6. En el ejemplo anterior la lista `combinations` ocupa alrededor de 140 MB. Este problema se vuelve más notorio cuando tenemos cadenas de caracteres más largas.
* Tiene una alta complejidad en tiempo. En el peor de los casos, la evaluación debe comparar la secuencia de caracteres contra cada una de las combinaciones del conjunto. Esto ocurre, por ejemplo, cuando buscamos una cadena que no está dentro de la lista de combinaciones válidas.

Para ilustrar esto último, veamos el tiempo que tarda hacer la comparación utilizando la librería `time`:

In [ ]:
import time

Evaluamos el tiempo:

In [ ]:
q = "1A2B3B"
t0 = time.time()
result = q in combinations
time_combs = time.time() - t0

Veamos cuánto tardó la búsqueda en segundos:

In [ ]:
print(time_combs)

Este resultado suele tardar cerca de medio segundo.

Ahora veamos qué ventaja nos da el mismo enfoque desde una **expresión regular**. Comencemos definiendo el patrón de la expresión regular para identificar las placas de los vehículos en Colombia:

In [ ]:
pat = r"[A-Z]{3,}\d{3,}"

Probemos los mismos dos ejemplos anteriores

Primero el que sí coincide:

In [ ]:
q = "ABC123"
print(re.match(pat, q) is not None)

Y ahora el que no coincide:

In [ ]:
q = "1A2B3C"
print(re.match(pat, q) is not None)

Veamos el tiempo que tarda la búsqueda con expresiones regulares:

In [ ]:
q = "1A2B3B"
t0 = time.time()
result = re.match(pat, q) is not None
time_regex = time.time() - t0

Veamos cuánto tardó la búsqueda en segundos:

In [ ]:
print(time_regex)

Veamos la relación entre el tiempo por combinaciones y el tiempo por expresión regular:

In [ ]:
ratio = time_combs / time_regex
print(ratio)

Como puede ver, el enfoque de expresiones regulares ofrece estas ventajas:

* Suele ser mucho más rápido en comparación con la estimación de todas las combinaciones.
* Lo único que debemos almacenar en memoria es la expresión regular; en este caso, la cadena `r"[A-Z]{3,}\d{3,}"`, y no alrededor de 17 millones de combinaciones posibles.

Veamos cómo podemos construir las expresiones regulares y qué opciones nos da la librería `re` para su manipulación.

## **2. Funciones con Expresiones Regulares**
---

Existen distintas funciones que podemos usar con expresiones regulares. Primero vamos a definir una expresión regular (más adelante veremos cómo construirlas):

In [ ]:
pat = r"\d{2}"

### **2.1 Compilación**
Podemos usar la función `compile` para obtener una versión compilada de la expresión regular. Generalmente, una expresión regular compilada es más rápida que una sin compilar:

In [ ]:
pat_comp = re.compile(pat)
print(pat_comp)

### **2.2 Coincidencia**
La función `match` nos permite validar si hay alguna coincidencia de la expresión regular con un texto dado. En el caso del ejemplo anterior, construimos una expresión regular (`pat_comp`) que representa dos dígitos seguidos.

Veamos un ejemplo sobre el siguiente texto:

In [ ]:
text = "13"

Ahora veamos la aplicación de la función `match`:

In [ ]:
match = re.match(pat_comp, text)
print(match)

El resultado es un objeto de tipo `Match` sobre el que podemos extraer información:

Con `match.start()` podemos obtener el índice donde inicia la coincidencia:

In [ ]:
print(match.start())

Con `match.end()` podemos obtener los índices donde termina la coincidencia:

In [ ]:
print(match.end())

Con `match.span()` podemos obtener los índices donde inicia y termina la coincidencia:

In [ ]:
print(match.span())

Con `match.string` podemos obtener el string de la coincidencia:

In [ ]:
print(match.string)

### **2.3 Búsqueda**
La función `search` nos permite buscar coincidencias de una expresión regular dentro de todo un texto.

Veamos un ejemplo con el siguiente texto:

In [ ]:
text = "mi cumpleaños es el 13 de marzo"

Veamos el resultado de la búsqueda:

In [ ]:
search = re.search(pat_comp, text)
print(search)

El resultado es un objeto de tipo `Match` con la primera coincidencia de la expresión regular dentro del texto.

### **2.4 Búsqueda múltiple**
La función `findall` nos permite encontrar todas las coincidencias de la expresión regular en un texto.

Veamos un ejemplo con el siguiente texto:

In [ ]:
text = "navidad es el 25 y año viejo el 31"

Veamos el resultado de la búsqueda múltiple

In [ ]:
multi_search = re.findall(pat_comp, text)
print(multi_search)

El resultado es una lista de strings con todas las coincidencias.

### **2.5 Búsqueda iterativa**
La función `finditer` permite encontrar objetos de tipo `Match` de todas las coincidencias.

Veamos un ejemplo:

In [ ]:
matches = list(re.finditer(pat_comp, text))
print(matches)

### **2.6 Sustitución**

La función `sub` permite reemplazar todas las coincidencias de una expresión regular por una secuencia dada dentro de un texto.

Veamos un ejemplo para reemplazar los dos dígitos detectados por la cadena `"XX"`:

In [ ]:
to_replace = "XX"

Veamos la sustitución:

In [ ]:
sub = re.sub(pat, to_replace, text)
print(sub)

### **2. 7 Separación**

La función `split` permite dividir una cadena de texto de acuerdo a una expresión regular.

Veamos un ejemplo donde separamos la siguiente cadena utilizando cualquier número como separador:

In [ ]:
to_split = "este1es2un3ejemplo4de5string"

Veamos la separación:

In [ ]:
vals = re.split(r"\d", to_split)
print(vals)

## **3. Identificadores de Caracteres**
---

La base de una expresión regular son los identificadores de caracteres que nos permiten definir patrones de cómo pueden variar las cadenas de caracteres. Una expresión regular puede construirse a partir de secuencias fijas (conjuntos de un único elemento) como mostramos a continuación:

In [ ]:
pat = re.compile(r"ejemplo")

Veamos las coincidencias de esta expresión regular dentro del siguiente texto:

In [ ]:
text = "ejemplo Ejemplo"

Veamos el resultado de un `findall` para ver las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Como puede ver, el resultado es únicamente la primera palabra, ya que disponemos de una expresión regular que representa un único conjunto y es igual a la palabra `ejemplo` en minúsculas.

Veamos cómo podemos extender la expresión regular para entender la segunda palabra con la siguiente expresión regular:

<img src="https://drive.google.com/uc?export=view&id=1Ot-nPYztYBJW1BLTpDk_cA269cO26VnP" width="60%">

In [ ]:
pat = re.compile(r"[eE]jemplo")

Ahora, veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

En este caso, usamos los corchetes cuadrados `[]` para crear el patrón de un caracter que puede tener dos posibles valores: `e` o `E`. Esto extiende la expresión regular a un conjunto de cardinalidad 2 con las siguientes combinaciones: `{"ejemplo", "Ejemplo"}`

Con los corchetes cuadrados podemos crear patrones para representar distintos caracteres, por ejemplo, podemos definir una expresión regular que extraiga cualquier secuencia de tamaño 1 de una vocal tildada:

In [ ]:
pat = re.compile(r"[éáíóú]")

Podemos usarla sobre el siguiente texto:

In [ ]:
text = "ayer perdí una canción de la memoria"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Adicional a esto, es posible abreviar la definición de grupos de caracteres usando el caracter `-`. Por ejemplo, la siguiente expresión regular define el conjunto de todas las letras minúsculas:

In [ ]:
pat = re.compile(r"[a-z]")

Veamos un ejemplo de su aplicación con el siguiente texto:

In [ ]:
text = "a b c d e f G H I J K"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Si deseamos que la expresión regular tenga en cuenta el caracter `-` para la búsqueda, o un backslash `\`, debemos escaparlos con un _backslash_ como mostramos a continuación:

In [ ]:
pat = re.compile(r"[\-\\]")

Veamos un ejemplo de su aplicación con el siguiente texto:

In [ ]:
text = r"este-ejemplo\es"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

También es posible definir secuencias de otros caracteres, típicamente se usan:

* `[a-z]`: todas las letras minúsculas.
* `[A-Z]`: todas las letras mayúsculas.
* `[0-9]`: todos los dígitos

Y de la misma forma, es posible combinarlos entre ellos. Veamos la siguiente expresión regular que define el conjunto de todas las letras minúsculas desde la `"a"` hasta la `"c"` y letras mayúsculas desde la `"D"` hasta la `"F"`:

In [ ]:
pat = re.compile(r"[a-cD-F]")

Veamos un ejemplo sobre el siguiente texto:

In [ ]:
text = r"abcDEFghijkLMN"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Existen abreviaciones para algunos conjuntos de caracteres; en la siguiente tabla encontrará los más comunes:

| Expresión Regular | Descripción | Ejemplo de Patrón | Ejemplo Match |
| --- | --- | --- | --- |
| `\d` | Un dígito | `file_\d\d` | `file_24` |
| `\w` | Caracter alfanumérico | `\w-\w\w\w` | `A-b_1` |
| `\s` | Espacio en blanco | `a\sb\sc` | `a b c` |
| `\D` | No dígito | `\D\D\D` | `abc` |
| `\W` | No alfanumérico | `\W\W\W` | `*=+` |
| `\S` | No es espacio | `\S\S\S` | `yoy` |
| `.` | Cualquier caracter | `....` | `Ae9=` |

También hay algunos caracteres especiales, los más comunes son:

| Expresión Regular | Descripción |
| --- | --- |
| `\t` | Tabulación |
| `\r` | Retorno del carro |
| `\n` | Salto de línea |
| `\f` | Salto de página |
| `\v` | Tabulación vertical|
| `\x` | Representa caracteres ascii, por ejemplo `\xA9` el cual representa © |
| `\u` | Formato unicode, por ejemplo `\u1F412` representa 🐒 |
| `^` | Inicio de una cadena |
| `$` | Final de una cadena |

## **4. Cuantificadores**
---
Los cuantificadores permiten representar las repeticiones de identificadores o secuencias de caracteres que van seguidos, por ejemplo, si quisiéramos detectar cualquier año del milenio pasado podríamos definir la siguiente expresión regular:

In [ ]:
pat = re.compile(r"1\d\d\d")

Como puede ver, definimos el identificador de dígitos 3 veces. Esto se puede simplificar con un cuantificador, así:

<img src="https://drive.google.com/uc?export=view&id=17EBQju0uImxW7BOAwmnE4WqXHhovkwMc" width="70%">

In [ ]:
pat = re.compile(r"1\d{3}")

En este caso, las llaves indican que un dígito `\d` se repite 3 veces, la expresión regular completa hace match con cualquier secuencia de números que inicie en 1 y esté seguido de 3 números.

Veamos un ejemplo sobre el siguiente texto:

In [ ]:
text = "1888 2002 288 2023 1039"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

También podemos definir rangos como cuantificadores. Por ejemplo, la siguiente expresión regular representa secuencias de máximo 3 dígitos:

In [ ]:
pat = re.compile(r"\d{,3}")

Veamos un ejemplo con el siguiente texto:

In [ ]:
text = "1 12 123 1234 12345 123456 1234567"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Como puede notar, el resultado incluye cadenas vacías `''`, esto se da por que se incluyen secuencias de dígitos de longitud 0. Podemos acotar el patrón a un mínimo de 1 dígito:

In [ ]:
pat = re.compile(r"\d{1,3}")

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

Tenemos algunas abreviaciones para los cuantificadores, estas se muestran en la siguiente tabla:

| Abreviación | Cuantificador | Descripción |
| --- | --- | --- |
| `?` | `{,1}` | Aparece 0 o una vez |
| `+` | `{1,}` | Aparece 1 o más veces |
| `*` | `{0,}` | Aparece 0 o más veces |

## **5. Operadores**
---

Tenemos algunos operadores que nos permiten modificar el comportamiento de una expresión regular, entre ellos, vimos el operador de rango `-`, el cual nos permite definir un rango de caracteres dentro de un contexto definido por corchetes cuadrados. Adicionalmente, tenemos estos operadores:

* `[^...]` representa el complemento de un conjunto de caracteres, por ejemplo, la siguiente expresión regular hace _match_ con cualquier secuencia (longitud mayor a 1) de caracteres que **no** sean dígitos:

In [ ]:
pat = re.compile(r"[^0-9]+")

Vamos a usar el siguiente texto como ejemplo:

In [ ]:
text = "hello 1234 bye 5467"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

* `|` representa la unión de dos expresiones regulares. Por ejemplo la siguiente expresión regular hace match con secuencias de 4 letras minúsculas o 3 letras mayúsculas:

In [ ]:
pat = re.compile(r"[a-z]{4}|[A-Z]{3}")

Vamos a usar el siguiente texto como ejemplo:

In [ ]:
text = "hola DIA pe del"

Veamos las coincidencias:

In [ ]:
search = re.findall(pat, text)
print(search)

## **6. Grupos**
---
Los grupos en expresiones regulares nos permiten estructurar y extraer información útil de los textos, un grupo se define por medio de los paréntesis `()`, así:

<img src="https://drive.google.com/uc?export=view&id=1cn3JUNGQzyfP7yIy71YP4j13zQD11bmW" width="60%">

Existen dos formas de definir los grupos:

### **6.1 Grupos posicionales**

Los grupos posicionales se definen de izquierda a derecha dentro de la expresión regular, por ejemplo, la siguiente expresión regular define dos grupos para capturar dos palabras cuya primera letra sea mayúscula y el resto minúsculas:

In [ ]:
pat = re.compile(r"([A-Z][a-z]+) ([A-Z][a-z]+)")

Podemos usarla para extraer nombres de un texto:

In [ ]:
text = "mi nombre es Pepe Perez y busco a Maria Belen"

Veamos el resultado de la búsqueda:

In [ ]:
search = list(re.finditer(pat, text))
print(search)

Como puede ver, el resultado de la búsqueda da lo mismo que el caso en el que no se usan grupos. No obstante, podemos usar el método `group` del objeto `Match` para extraer la información de cada grupo, por ejemplo, los primeros nombres:

In [ ]:
print([match.group(1) for match in search])

También podemos extraer los apellidos:

In [ ]:
print([match.group(2) for match in search])

### **6.2 Grupos nombrados**


<img src="https://drive.google.com/uc?export=view&id=1jZilAJSXdeCw2flg7cnhyUooiiI-w0sK" width="60%">

Los grupos nombrados se crean usando la notación `(?P<tag>...)`. Por ejemplo, podemos definir la misma expresión regular de arriba con la diferencia de que ahora damos nombre a los dos grupos:

In [ ]:
pat = re.compile(r"(?P<nombre>[A-Z][a-z]+) (?P<apellido>[A-Z][a-z]+)")

Veamos el resultado de la búsqueda:

In [ ]:
search = list(re.finditer(pat, text))
print(search)

Así mismo, podemos extraer los nombres y apellidos usando la etiqueta asignada a cada grupo, por ejemplo, los nombres:

In [ ]:
print([match.group("nombre") for match in search])

In [ ]:
print([match.group("apellido") for match in search])

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información para profundizar en los temas vistos en este taller guiado:

* [Regex en Python](https://docs.python.org/3/howto/regex.html)
* [Learn and test regex](https://regexr.com/)
* [Acerca de las expresiones regulares](https://support.google.com/analytics/answer/1034324?hl=es)

## Créditos
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).   
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*